In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Synthetic data for demonstration - replace with actual model features
np.random.seed(42)
torch.manual_seed(42)

num_samples = 1000
textual_features = np.random.rand(num_samples, 10)   # e.g., 10-dimensional textual features
audio_features = np.random.rand(num_samples, 10)     # e.g., 10-dimensional audio features
facial_features = np.random.rand(num_samples, 10)    # e.g., 10-dimensional facial features
labels = np.random.randint(0, 2, num_samples)        # Binary labels (0 or 1)

# Combine features into a single dataset
features = np.hstack([textual_features, audio_features, facial_features])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the neural network fusion model
class FusionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FusionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.output(x))
        return x

# Hyperparameters
input_size = X_train.shape[1]  # 30 (10 for each modality)
hidden_size = 64
num_epochs = 50
learning_rate = 0.001

# Initialize the model, loss function, and optimizer
model = FusionModel(input_size=input_size, hidden_size=hidden_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = (y_pred > 0.5).float()  # Apply threshold for binary classification
    accuracy = accuracy_score(y_test_tensor, y_pred_class)
    print("Fusion Model Accuracy:", accuracy)
